### Podczas testów klinicznych szczepionki na COVID19 firma Pfizer wskazała, że spośród przetestowanej grupy, której podano szczepionkę 9 na 21500 zakaziło się wirusem, natomiast firma Moderna wskazała, że jej szczepionka jest skuteczniejsza ponieważ na 15000 badanych jedynie 5 osób zostało zakażonych. Spróbuj odpowiedzieć na pytania:

    Jaka jest średnia skuteczność, odchylenie standardowe rozkładu obu szczepionek i oszacuj (metodą próbkowania bootstrapowego) jaki jest przedział ufności (stwórz wektor binarny zawierający tyle elementów ile osób brało udział w badaniach w którym 0 odpowiada osobie, dla której szczepionka nie zadziałała)

In [47]:
import numpy as np
import pandas as pd
import sklearn.utils
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [48]:
pfizer = np.ones(21500)
pfizer[:9] = 0
# 9 / 21_500


moderna = np.ones(15_00)
moderna[:5] = 0
# 5 / 15_000



In [49]:
print('Średnia skuteczność pfizer: ', ( np.mean(pfizer)) * 100)
print('Średnia skuteczność moderna: ', ( np.mean(moderna)) * 100)

Średnia skuteczność pfizer:  99.95813953488371
Średnia skuteczność moderna:  99.66666666666667


In [50]:
print('Odchylenie standardowe pfizer: ', np.std(pfizer))
print('Odchylenie standardowe moderna: ', np.std(moderna))

Odchylenie standardowe pfizer:  0.02045554744583521
Odchylenie standardowe moderna:  0.05763872155263527


In [51]:


means = []
n_boot = 10_000
for i in range(1, n_boot):
    boot = sklearn.utils.resample(pfizer, replace=True, n_samples=len(pfizer))
    means.append(np.mean(boot))
means = np.array(means)
means.sort()

print(f'Przedział ufności pfeizer: {means[int(n_boot * .025)]}, {means[int(n_boot * .975)]}')

means = []
for i in range(1, n_boot):
    boot = sklearn.utils.resample(moderna, replace=True, n_samples=len(moderna))
    means.append(np.mean(boot))
means = np.array(means)
means.sort()

print(f'Przedział ufności moderna: {means[int(n_boot * .025)]}, {means[int(n_boot * .975)]}')


Przedział ufności pfeizer: 0.9993023255813953, 0.9998139534883721
Przedział ufności moderna: 0.9933333333333333, 0.9993333333333333


### Zweryfikuj czy prawdą jest, że jedna szczepionka jest skuteczniejsza od drugiej?

In [52]:
(stats, p_value) = stats.ttest_ind(pfizer, moderna, equal_var = False)

stats, p_value

(1.949332938853141, 0.051438727217599255)

Otrzymane p-value większe od założonego na poziomie 0.05.

Brak przesłanek za odrzuceniem hipotezy zerowej.

### Wyznacz aktualną moc testu

In [53]:
from statsmodels.stats.power import TTestIndPower

effect = (np.mean(pfizer) - np.mean(moderna)) / (np.std(pfizer) + np.std(moderna)) / 2
analysis = TTestIndPower()
result = analysis.solve_power(effect, power=None, nobs1=len(pfizer), ratio=len(moderna)/len(pfizer), alpha=0.05)

print(f'Aktualna moc testu: {result}')

result = analysis.solve_power(effect, power=None, nobs1=len(moderna), ratio=len(pfizer)/len(moderna), alpha=0.05)

print(f'Aktualna moc testu: {result}')

Aktualna moc testu: 0.10754055618168767
Aktualna moc testu: 0.10754055618168767


In [54]:
power = 0.60
result = analysis.solve_power(effect, power=power, nobs1=None, ratio=len(moderna)/len(pfizer), alpha=0.05)


print(f'Aby osiągnąć moc testu na poziomie {power * 100}% należy zebrać: {result} próbek')

Aby osiągnąć moc testu na poziomie 60.0% należy zebrać: 215681.01351369993 próbek
